In [1]:
! pip install -Uqq fastbook
from fastbook import *

In [2]:
path = untar_data(URLs.ML_100k)

In [3]:
path.ls()

(#23) [Path('/Users/modsoussi/.fastai/data/ml-100k/u.item'),Path('/Users/modsoussi/.fastai/data/ml-100k/u3.test'),Path('/Users/modsoussi/.fastai/data/ml-100k/u1.base'),Path('/Users/modsoussi/.fastai/data/ml-100k/u.info'),Path('/Users/modsoussi/.fastai/data/ml-100k/u2.test'),Path('/Users/modsoussi/.fastai/data/ml-100k/u5.test'),Path('/Users/modsoussi/.fastai/data/ml-100k/u.genre'),Path('/Users/modsoussi/.fastai/data/ml-100k/ub.test'),Path('/Users/modsoussi/.fastai/data/ml-100k/ua.base'),Path('/Users/modsoussi/.fastai/data/ml-100k/u.data')...]

In [4]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                 names=['user', 'movie', 'rating', 'timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
movies = pd.read_csv(path/'u.item', delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=['movie', 'title'], header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [6]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [7]:
from fastai.collab import *

dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,542,My Left Foot (1989),4
1,422,Event Horizon (1997),3
2,311,"African Queen, The (1951)",4
3,595,Face/Off (1997),4
4,617,Evil Dead II (1987),1
5,158,Jurassic Park (1993),5
6,836,Chasing Amy (1997),3
7,474,Emma (1996),3
8,466,Jackie Chan's First Strike (1996),3
9,554,Scream (1996),3


In [8]:
u_factors = torch.nn.Embedding(25, 5)
i_factors = torch.nn.Embedding(15, 5)

idx = torch.tensor([[1,2], [4, 3]])

In [9]:
u_emb, i_emb = u_factors(idx[:, 0]), i_factors(idx[:, 1])
u_emb, i_emb

(tensor([[-0.5195,  0.7613, -0.4365,  0.1365,  1.3300],
         [-0.1345, -1.5280,  0.0200, -0.1748, -0.1233]], grad_fn=<EmbeddingBackward0>),
 tensor([[-1.0512,  1.1215,  1.1948, -0.0144,  0.7229],
         [ 0.7811, -0.8685, -0.0818, -0.6450, -1.0731]], grad_fn=<EmbeddingBackward0>))

In [10]:
(u_emb * i_emb).sum(dim=1)

tensor([1.8378, 1.4654], grad_fn=<SumBackward1>)

In [11]:
class Collab(torch.nn.Module):
  def __init__(self, num_users, num_items, num_factors):
    super().__init__()
    self.user_factors = torch.nn.Embedding(num_users, num_factors)
    self.item_factors = torch.nn.Embedding(num_items, num_factors)
  
  def forward(self, x):
    users = self.user_factors(x[:, 0])
    items = self.item_factors(x[:, 1])
    return (users*items).sum(dim=1)

In [12]:
x, y = dls.one_batch()
x.shape, y.shape

(torch.Size([64, 2]), torch.Size([64, 1]))

In [13]:
x[:5], y[:5]

(tensor([[ 661,  659],
         [ 861,  730],
         [ 766,    8],
         [  13, 1153],
         [ 429, 1185]]),
 tensor([[4],
         [4],
         [4],
         [1],
         [3]], dtype=torch.int8))

In [14]:
num_users = len(dls.classes['user'])
num_movies = len(dls.classes['title'])

In [15]:
model = Collab(num_users, num_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [16]:
learn.fit_one_cycle(5, 5e-3)

In [17]:
class Collab(torch.nn.Module):
  def __init__(self, num_users, num_items, num_factors, y_range=(0, 5.5)):
    super().__init__()
    self.user_factors = torch.nn.Embedding(num_users, num_factors)
    self.item_factors = torch.nn.Embedding(num_items, num_factors)
    self.y_range = y_range
    
  def forward(self, x):
    users = self.user_factors(x[:, 0])
    items = self.item_factors(x[:, 1])
    return sigmoid_range((users * items).sum(dim=1), *self.y_range)
    

In [18]:
model = Collab(num_users, num_movies,50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [19]:
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,7.621876,7.591930,00:07
1,6.635986,7.230602,00:07
2,5.344331,6.808065,00:06
3,4.486286,6.566132,00:07
4,4.125020,6.522335,00:07


In [20]:
class CollabBias(torch.nn.Module):
  def __init__(self, num_users, num_items, num_factors, y_range=(0, 5.5)):
    super().__init__()
    self.user_factors = Embedding(num_users, num_factors)
    self.user_bias = Embedding(num_users, 1)
    self.item_factors = Embedding(num_items, num_factors)
    self.item_bias = Embedding(num_items, 1)
    self.y_range = y_range
    
  def forward(self, x):
    users = self.user_factors(x[:, 0])
    items = self.item_factors(x[:, 1])
    res = (users * items).sum(dim=1, keepdim=True)
    res += self.user_bias(x[:, 0]) + self.item_bias(x[:, 1])
    return sigmoid_range(res, *self.y_range)

In [21]:
model = CollabBias(num_users, num_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [22]:
learn.fit_one_cycle(5, 5e-3, wd=.1)

epoch,train_loss,valid_loss,time
0,0.941235,0.954500,00:10
1,0.882794,0.875728,00:09
2,0.720068,0.832267,00:09
3,0.603367,0.820231,00:09
4,0.475883,0.821274,00:09


In [23]:
movie_bias = learn.model.item_bias.weight.squeeze()
movie_bias

tensor([-0.0005, -0.1357,  0.0250,  ..., -0.0296,  0.0596,  0.0276], device='mps:0', grad_fn=<SqueezeBackward0>)

In [24]:
idx = movie_bias.argsort()[:5]
[dls.classes['title'][i] for i in idx]

['Children of the Corn: The Gathering (1996)',
 'Lawnmower Man 2: Beyond Cyberspace (1996)',
 'Robocop 3 (1993)',
 'Beautician and the Beast, The (1997)',
 'Crow: City of Angels, The (1996)']

In [25]:
idx = movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idx]

['L.A. Confidential (1997)',
 'Titanic (1997)',
 'Silence of the Lambs, The (1991)',
 "Schindler's List (1993)",
 'Star Wars (1977)']

In [26]:
class CollabNN(torch.nn.Module):
  def __init__(self, user_sz, item_sz, y_range=(0, 5.5), n_acts=100):
    super().__init__()
    self.user_factors = torch.nn.Embedding(*user_sz)
    self.item_factors = torch.nn.Embedding(*item_sz)
    self.layers = torch.nn.Sequential(
      torch.nn.Linear(user_sz[1]+ item_sz[1], n_acts),
      torch.nn.ReLU(),
      torch.nn.Linear(n_acts, 1)
    )
    self.y_range = y_range
    
  def forward(self, x):
    embs = self.user_factors(x[:, 0]), self.item_factors(x[:, 1])
    x = self.layers(torch.cat(embs, dim=1))
    return sigmoid_range(x, *self.y_range)

In [27]:
from fastai.collab import get_emb_sz, collab_learner

In [28]:
embs = get_emb_sz(dls)
embs

[(944, 74), (1665, 102)]

In [29]:
model = CollabNN(*embs)

In [30]:
learner = Learner(dls, model, loss_func=MSELossFlat())
learner.fit_one_cycle(5, 5e-3, wd=.1)

epoch,train_loss,valid_loss,time
0,0.981195,1.019254,00:13
1,0.948425,0.927893,00:11
2,0.899064,0.902260,00:11
3,0.791158,0.877160,00:11
4,0.733255,0.881398,00:11
